In [25]:
%load_ext autoreload

%autoreload 2

In [1]:
!rm -rf loracraft
!git clone https://github.com/PPierzc/loracraft.git
!mv loracraft loracraft-git
!mv loracraft-git/* .
!rm -rf loracraft-git

Cloning into 'loracraft'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 36 (delta 12), reused 29 (delta 7), pack-reused 0
Receiving objects: 100% (36/36), 9.03 KiB | 2.26 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [1]:
import json
import torch
import os
from loracraft.pipeline import get_pipeline
from loracraft.generator_utils import save_image

In [2]:
with open ("/content/lora_db.json", 'r') as f:
    lora_db = json.load(f)

In [3]:
config = {
    "loras": ["SewingDoll", "CinematicPainting"],
    "weights": [0.7, 0.3],
    "prompts": ["Little nightmares"],
    "trials": 1,

    'images_path': '/content/output/sewingdoll-cinematic/',
    'seed': 42,
    'pipeline_kwargs': {
        'num_inference_steps': 1,
        "guidance_scale": 0,
    }
}

In [4]:
## create pipeline
pipe = get_pipeline()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
import wandb

In [6]:
wandb.login()

wandb: Currently logged in as: anxious-insecure. Use `wandb login --relogin` to force relogin


True

In [7]:
# Download required LoRAs
for lora in config["loras"]:

    lora_path = f"./{lora}.safetensors"
    lora_wget = lora_db[lora]["wget_link"]
    command = f'wget -O {lora_path} "{lora_wget}'

    if not os.path.exists(lora_path):
        ! wget -O $lora_path "$lora_wget"

    pipe.load_lora_weights(
        lora_path,
        adapter_name=lora
    )

In [8]:
pipe.set_adapters(config["loras"], adapter_weights=config["weights"])

In [9]:
os.makedirs(config['images_path'], exist_ok=True)

run = wandb.init(
    entity='ppierzc',
    project='loracraft',
    config={"config": config['images_path']}
)

for trial in range(config['trials']):
    for prompt in config['prompts']:
        image = pipe(
            prompt=prompt,
            generator=torch.manual_seed(config['seed']),
            **config['pipeline_kwargs']
        ).images[0]
        save_image(image, config, prompt=prompt, trial=trial, run=run)

  0%|          | 0/1 [00:00<?, ?it/s]